<a href="https://colab.research.google.com/github/makhlufiaero338/tugas-machine-learning/blob/main/week12/adam_CIFAR_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torch.optim.lr_scheduler import StepLR
import torch.nn.functional as F

In [2]:
# Early Stopping Callback
class EarlyStopping:
    def __init__(self, patience=5, delta=0):
        self.patience = patience
        self.delta = delta
        self.best_loss = float('inf')
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

In [3]:
# Load CIFAR-10 Dataset
def load_data(batch_size=64):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

    train_size = int(0.8 * len(train_dataset))
    val_size = len(train_dataset) - train_size
    train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader

In [4]:
# Define CNN Model
class CNN(nn.Module):
    def __init__(self, kernel_size=3, pooling='max'):
        super(CNN, self).__init__()
        if pooling == 'max':
            pool = nn.MaxPool2d(2, 2)
        else:
            pool = nn.AvgPool2d(2, 2)

        self.conv1 = nn.Conv2d(3, 32, kernel_size=kernel_size, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=kernel_size, padding=1)
        self.pool = pool
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [5]:
# Train and Validate Model
def train_model(model, train_loader, val_loader, optimizer, scheduler, criterion, num_epochs=50, patience=5):
    early_stopping = EarlyStopping(patience=patience)

    train_losses, val_losses = [], []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        train_loss = running_loss / len(train_loader)
        train_losses.append(train_loss)

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        val_loss /= len(val_loader)
        val_losses.append(val_loss)

        scheduler.step(val_loss)

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

        early_stopping(val_loss)
        if early_stopping.early_stop:
            print("Early stopping triggered.")
            break

    return train_losses, val_losses

In [6]:
# Evaluate Model
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    return accuracy


In [7]:
# Main Execution
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_loader, val_loader, test_loader = load_data(batch_size=64)

# Configurations for Adam
kernel_size = 3
pooling = 'max'
model = CNN(kernel_size=kernel_size, pooling=pooling).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=10, gamma=0.1)

train_losses, val_losses = train_model(
    model, train_loader, val_loader, optimizer, scheduler, criterion, num_epochs=50, patience=5
)
accuracy = evaluate_model(model, test_loader)

100%|██████████| 170M/170M [00:03<00:00, 53.6MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:240: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch 1/50, Train Loss: 1.3988, Val Loss: 1.1509
Epoch 2/50, Train Loss: 1.0267, Val Loss: 1.0577
Epoch 3/50, Train Loss: 0.8669, Val Loss: 0.9464
Epoch 4/50, Train Loss: 0.7555, Val Loss: 0.9229
Epoch 5/50, Train Loss: 0.6531, Val Loss: 0.8874
Epoch 6/50, Train Loss: 0.5628, Val Loss: 0.9264
Epoch 7/50, Train Loss: 0.4710, Val Loss: 0.9463
Epoch 8/50, Train Loss: 0.3952, Val Loss: 0.9763
Epoch 9/50, Train Loss: 0.3162, Val Loss: 1.0479
Epoch 10/50, Train Loss: 0.2528, Val Loss: 1.2013
Early stopping triggered.
Test Accuracy: 70.44%
